Team Members: 
Anisha Alluru, 
Khyathi Balusu, 
Ravikiran Bobba, 
Manas Rai, 
Vishal Ramachandran


## Colab Setup

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Data Scraping

In [0]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chrom

In [0]:
import pandas as pd
# webdriver provides capabilities for navigating to web pages
driver = webdriver.Chrome('chromedriver', options=chrome_options)

In [0]:
# scraping and storing links
links = pd.DataFrame(columns=['beer','link'])

driver.get('https://www.beeradvocate.com/beer/top-rated/')

for i in range(250):

  links.loc[i,'link'] = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr[' + str(i+2) + ']/td[2]/a')[0].get_attribute('href')
  links.loc[i,'beer'] = driver.find_elements_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/table/tbody/tr[' + str(i+2) + ']/td[2]/a/b')[0].text

In [0]:
# scraping reviews going through all the links
df = pd.DataFrame(columns = ['product_name','product_review','user_rating'])
for i in range(len(links)):
  
  driver.get(links.iloc[i,1])
  for j in range(25):
    try:
      product_review = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+ str(j+1) +']/div[2]').text
      df.loc[25*i+j,'product_name'] = links.iloc[i,0]
      df.loc[25*i+j,'product_review'] = product_review
      df.loc[25*i+j,'user_rating'] = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[2]/div[2]/div/div/div[3]/div/div/div[2]/div[8]/div/div['+ str(j+1) +']/div[2]/span[1]').text
    except:
      break

In [0]:
import re
# taking out only the review content from the scrapped data
df.iloc[:,1] = df.iloc[:,1].map(lambda x:re.split('\n\n',x)[1:-2])
df.iloc[:,1] = df.iloc[:,1].map(lambda x:" ".join(x))

In [0]:
df.to_csv('scrapped_beer_data.csv')

In [0]:
df.head()

,product_name,product_review,user_rating
0,Kentucky Brunch Brand Stout,"2016 Silver Wax. Aroma has whiskey, maple, tof...",4.80
1,Kentucky Brunch Brand Stout,The beer pours Pitch Black with a frothy tan h...,4.74
2,Kentucky Brunch Brand Stout,Probably the smoothest beer I have ever had. S...,4.68
3,Kentucky Brunch Brand Stout,"Dark black, very thick, a little bit of tan he...",5.00
4,Kentucky Brunch Brand Stout,Poured black as ink with thin ruby edges at 58...,4.97


## Similarity Analysis

In [0]:
# https://medium.com/better-programming/the-beginners-guide-to-similarity-matching-using-spacy-782fc2922f7c

In [0]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg

nlp = en_core_web_lg.load()
#nlp = spacy.load('en_core_web_lg')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
# sample check
doc1 = nlp("How do I turn sound on/off?")
doc2 = nlp("How do I obtain a pet?")
doc1.similarity(doc2)

0.8680367210908022

In [0]:
# selecting top 3 attributes - All 3 are significantly present in the reviews - verified in the scrapped in Excel
attributes = 'Fruity Robust Balanced'

In [0]:
df.loc[:,"similarity"] = df.iloc[:,1].map(lambda x:nlp(x).similarity(nlp(attributes)))

In [0]:
top_300 = df.sort_values(by = 'similarity', ascending=False)[:300]

In [0]:
top_300

,product_name,product_review,user_rating,similarity
732,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,4.50,0.715458
4105,Saison Bernice,Tasted from bottle. Classic farmhouse saison. ...,4.44,0.714199
1108,Double Galaxy,"Soft gold, foamy head. Aroma has underripe mel...",4.37,0.713743
3006,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,4.40,0.710848
3706,Lou Pepe - Gueuze,Clear orange pour. Nose full of citrus includi...,4.50,0.709529
...,...,...,...,...
5043,Rodenbach Caractère Rouge,Red/amber/lite brown pour; quick thin white ca...,4.85,0.647869
2165,Alter Ego,16oz can dated 3/19/19. Pours murky gold with ...,4.04,0.647765
4479,La Fosse,"Blend #3. A: Pours a fizzy, light pale orange ...",4.42,0.647708
4266,Stickee Monkee,Bottle: Poured a dark burgundy color ale with ...,3.75,0.647680


## Sentiment Analysis

In [0]:
# https://medium.com/analytics-vidhya/simplifying-social-media-sentiment-analysis-using-vader-in-python-f9e6ec6fc52f

In [0]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 2.8MB/s 


In [0]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [0]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

In [0]:
# sample check
sentiment_analyzer_scores("The phone is super COOL!! :)")

0.8889

In [0]:
top_300.loc[:,'sentiment_score'] = top_300.loc[:,'product_review'].map(lambda x:sentiment_analyzer_scores(x))

In [0]:
top_300.head()

,product_name,product_review,user_rating,similarity,sentiment_score
1700,Doppelganger,"Dark, murky orange in color with an unfiltered...",4.43,0.718292,0.8979
732,Duck Duck Gooze,On tap at Beavertown Extravaganza. Aroma has s...,4.5,0.715458,0.6908
4129,Saison Bernice,Tasted from bottle. Classic farmhouse saison. ...,4.44,0.714199,0.5106
1108,Double Galaxy,"Soft gold, foamy head. Aroma has underripe mel...",4.37,0.713743,-0.0377
3032,Pseudo Sue,Murky creamsicle orange in color. Poured with ...,4.4,0.710848,0.9286


In [0]:
top_300['user_rating'] = top_300['user_rating'].astype(float)

## Question E: Our Recommendation

In [0]:
# We will recommend top 3 products based on highest sentiment score
top_300.groupby('product_name')['user_rating','similarity','sentiment_score'].mean().reset_index().sort_values(by = 'sentiment_score', ascending = False).head(3)

,product_name,user_rating,similarity,sentiment_score
45,Double Dry Hopped Congress Street,4.15,0.658891,0.9958
7,Art,4.67,0.654545,0.9951
94,Last Snow,4.45,0.655022,0.9887


Above mentioned are the three beers that we would recommend to a person if he is looking for a fruity, robust and a balanced beer

# Question F: User Rating Based Recommendation

Performing on thw whole dataset

In [0]:
df['user_rating'] = df['user_rating'].astype(float)

In [0]:
df.loc[:,'sentiment_score'] = df.loc[:,'product_review'].map(lambda x:sentiment_analyzer_scores(x))
df.loc[:,"similarity"] = df.iloc[:,1].map(lambda x:nlp(x).similarity(nlp(attributes)))

In [0]:
#We will recommend top 3 products based on highest average user ratings
df.groupby('product_name')['user_rating','sentiment_score','similarity'].mean().reset_index().sort_values(by = 'user_rating', ascending = False).head(3)

,product_name,user_rating,sentiment_score,similarity
55,Chemtrailmix,4.816667,0.883917,0.533751
149,Kentucky Brunch Brand Stout,4.812000,0.787348,0.518901
140,It Was All A Dream,4.785000,0.600915,0.534646


If we simply recommend based on maximum user rating without considering the sentiment score or similarity score, we will miss out on a relevant recommendation. As seen in the above table of top 3 recommendations based on just user rating, the sentiment and similarity score for the selected attributes is relatively low even though the user rating is high. For e.g. - Chemtrailmix is more a spiced beer which falls under Hoppy category and it is definitely not the attribute which we selected.

The previous beer recommendations made were based on similarity with the 3 selected attributes (Fruity, Robust and Balanced) and the general sentiment based on the comments. These recommendations will be best suited for a customer who keeps these three attributes as their top priority as we can see that both similarity and sentiment scores are very high.

More generic and simply user rating based recommendation would be best suited for customers who do not have any attribute preference or their preference is not known.
